In [1]:
import re
import nltk
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from nltk.corpus import stopwords
from collections import defaultdict
from spellchecker import SpellChecker
from kedro.pipeline.decorators import log_time
import emoji
import spacy
import textacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

ModuleNotFoundError: No module named 'emoji'

In [2]:
reviews = io.load('reviews_master')

DataSetNotFoundError: DataSet 'reviews_master' not found in the catalog

### Rules for Text Cleaning:
- Short hand abbrev:
    + bc -> because
- Repeating words (sooo):
    + regex??

In [3]:
# add special case rule
from spacy.attrs import ORTH, LEMMA, POS

special_case = [{ORTH: u"bc", LEMMA: u"because", POS: u"CONJ"}]
nlp.tokenizer.add_special_case(u"bc", special_case)

In [4]:
def clean_text(text):
    text = emoji.demojize(text)
    return textacy.preprocess.preprocess_text(text, False, 
                                              lowercase=True, no_urls=True, 
                                              no_emails=True, no_phone_numbers=True, 
                                              no_numbers=True, no_currency_symbols=False, 
                                              no_punct=True, no_contractions=False, 
                                              no_accents=True)

In [5]:
reviews['clean_text'] = reviews['review_text'].progress_apply(clean_text)

In [6]:
reviews.head()

,product_id,author_age,review_title,review_text,star_rating,recommend_flag,upvotes,product_category_division,product_category_department,product_category_class,clean_text
id,,,,,,,,,,,
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,intimates,intimate,intimates,absolutely wonderful silky and sexy and comfor...
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,general,dresses,dresses,love this dress it s sooo pretty i happened to...
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,general,dresses,dresses,i had such high hopes for this dress and reall...
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,general petite,bottoms,pants,i love love love this jumpsuit it s fun flirty...
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,general,tops,blouses,this shirt is very flattering to all due to th...


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
analyzer = TfidfVectorizer().build_analyzer()
stemmer = nltk.PorterStemmer()
def stemmed_words(doc):
    return list(map(stemmer.stem, analyzer(doc)) )

vectorizer = TfidfVectorizer(ngram_range=(1,2), 
                             stop_words='english',
                             analyzer=stemmed_words,
                             norm='l1')
corpus = list(reviews['clean_text'])

X = vectorizer.fit_transform(tqdm(corpus, desc='Calculating TF-iDF', total=len(reviews)))

In [46]:
#12613 unique lemmas
#9694 unique Porter stems
#8190 unique Lancaster stems
print(X.shape)

(22641, 9694)


Will go with Porter stemming because not much difference to Lancaster while retaining more meaning of words

Only considers top 3000 stems

In [47]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), 
                             stop_words='english',
                             analyzer=stemmed_words,
                             max_features=3000,
                             norm='l1')
corpus = list(reviews['clean_text'])

X = vectorizer.fit_transform(tqdm(corpus, desc='Calculating TF-iDF', total=len(reviews)))

In [48]:
import pickle
from kedro.io import PickleLocalDataSet

data_set = PickleLocalDataSet(filepath="data.pkl",
                              backend='pickle',
                              load_args=None,
                              save_args=None)
data_set.save(X)